# Modelo

In [1]:
import pandas as pd
import numpy as np

In [2]:
uri = 'bolt://localhost:7687'
user = 'neo4j'
psw = 'password'

In [3]:
x_train = pd.read_csv('data/x_train.csv')
y_train = pd.read_csv('data/y_train.csv')
x_val = pd.read_csv('data/x_val.csv')
y_val = pd.read_csv('data/y_val.csv')
x_test = pd.read_csv('data/x_test.csv')
y_test = pd.read_csv('data/y_test.csv')
features = ['cat_based', 'cat_cnt', 'user_based', 'user_cnt',
            'adamic_adar', 'resource_allocation', 'link_cnt', 'cat_avg', 'user_avg',
            'adar_avg', 'ra_avg']

In [3]:
len(y_train), len(y_val), len(y_test)

(706, 151, 152)

In [162]:
train = [i for i in range(706)]
val = [706 + i for i in range(151)]
test = [856 + i for i in range(152)]

### Normalizador

In [96]:
df = pd.concat([x_train, x_val, x_test])
df.head()

,user_id,podcast_id,cat_based,cat_cnt,user_based,user_cnt,adamic_adar,resource_allocation,link_cnt,cat_avg,user_avg,adar_avg,ra_avg
0,426DDBF8D527E3A,b69677f9f694bd3566eaf9fff0343784,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0,0.00,0.0,0.000000,0.000000
1,E2FC7EABA5761EC,b5cf7d57fdf09b9e4177a0afb8cd467f,7.0,7.0,0.0,0.0,2.998319,0.717647,5.0,1.00,0.0,0.599664,0.143529
2,AC072D3BF1CB749,b4c3c3ebdd76e284f7d9fa358ac82030,10.0,2.0,45.0,3.0,10.440813,2.436210,2.0,5.00,15.0,5.220406,1.218105
3,5C82AAF7008477B,b1a3eb2aa8e82ecbe9c91ed9a963c362,60.0,16.0,52.0,4.0,26.122381,4.198098,13.0,3.75,13.0,2.009414,0.322931
4,5D7ADA74FE6E018,dfa087abaf0dd92207c499a9ed5e4451,20.0,4.0,0.0,0.0,7.104152,1.361685,5.0,5.00,0.0,1.420830,0.272337


In [132]:
max_df = df.groupby('user_id')[features].max()
max_df.sum()

cat_based              1322.000000
cat_cnt                 282.000000
user_based             6443.000000
user_cnt                443.000000
adamic_adar            1952.829911
resource_allocation     575.056886
link_cnt                291.000000
cat_avg                 278.892857
user_avg                708.033333
adar_avg                829.161883
ra_avg                  263.489960
dtype: float64

In [133]:
df[df['user_id'] == '14F47B6A810BA33'][features]/0.

,cat_based,cat_cnt,user_based,user_cnt,adamic_adar,resource_allocation,link_cnt,cat_avg,user_avg,adar_avg,ra_avg
689,inf,inf,NaN,NaN,inf,inf,inf,inf,NaN,inf,inf
139,inf,inf,NaN,NaN,inf,inf,inf,inf,NaN,inf,inf


In [143]:

def norm_row(row, max_df):
    try:
        return row[features].values / max_df.loc[row['user_id']].values
    except:
        return [0] * len(features)

def norm_by_user(df, max_df):
    new_df = pd.DataFrame()
    new_df[features] = df.apply(lambda row: norm_row(row, max_df), axis = 1, result_type='expand')
    return new_df.fillna(0)

In [147]:
norm = norm_by_user(df, max_df)
norm.describe()

,cat_based,cat_cnt,user_based,user_cnt,adamic_adar,resource_allocation,link_cnt,cat_avg,user_avg,adar_avg,ra_avg
count,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000,1009.000000
mean,0.443932,0.482266,0.367770,0.382974,0.430134,0.393652,0.658077,0.524905,0.460782,0.430134,0.393652
std,0.415557,0.432655,0.420172,0.435045,0.407566,0.405149,0.474589,0.460159,0.467875,0.407566,0.405149
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,0.555556,0.042963,0.044444,0.426870,0.247035,1.000000,0.800000,0.466667,0.426870,0.247035
75%,0.850000,1.000000,0.831461,1.000000,0.868866,0.849875,1.000000,1.000000,1.000000,0.868866,0.849875
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [158]:
norm.iloc[1000]

cat_based              0.0
cat_cnt                0.0
user_based             0.0
user_cnt               0.0
adamic_adar            0.0
resource_allocation    0.0
link_cnt               0.0
cat_avg                0.0
user_avg               0.0
adar_avg               0.0
ra_avg                 0.0
Name: 143, dtype: float64

In [163]:
x_train_norm = norm.iloc[train].copy()
x_val_norm = norm.iloc[val].copy()
x_test_norm = norm.iloc[test].copy()

In [196]:
x_train_norm.to_csv('data/x_train_norm.csv', index=False)
x_val_norm.to_csv('data/x_val_norm.csv', index=False)
x_test_norm.to_csv('data/x_test_norm.csv', index=False)

In [114]:
x_train_norm = pd.read_csv('data/x_train_norm.csv')
x_val_norm = pd.read_csv('data/x_val_norm.csv')

In [128]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.5)
lr.fit(x_train_norm[features], y_train.values.reshape(1,-1)[0])

LogisticRegression(C=0.5)

In [129]:
pred = lr.predict(x_val_norm[features])

In [130]:
(pred == y_val.values).mean(), y_val.values.mean()

(0.7665453269593439, 0.7814569536423841)

In [127]:
pred.mean()

1.0

In [164]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', C=0.5)
lr.fit(x_train[features], y_train.values.reshape(1,-1)[0])

LogisticRegression(C=0.5, solver='liblinear')

In [173]:
pred = lr.predict_proba(x_val[features])
(pred == y_val.values).mean(), y_val.values.mean()

(0.0, 0.7814569536423841)

In [175]:
pred[:,1], y_val.values.reshape(1,-1)

(array([0.9765784 , 0.83735914, 0.94817315, 0.97539038, 0.99715436,
        0.99068868, 0.56446921, 0.77782803, 0.52439113, 0.99066613,
        0.9677561 , 0.90470777, 0.99501969, 0.67328874, 0.99066613,
        0.52116607, 0.99511287, 0.67328874, 0.89421599, 0.97419214,
        0.52116607, 0.81395661, 0.97086448, 0.02672527, 0.99511287,
        0.52116607, 0.67297358, 0.99489216, 0.52439113, 0.92863467,
        0.55550799, 0.96569192, 0.89386238, 0.99999961, 0.72749088,
        0.86758888, 0.9552558 , 0.97805946, 0.99362465, 0.99980085,
        0.83735914, 0.9525641 , 0.98594446, 0.82032334, 0.98481781,
        0.02622049, 0.64092466, 0.52116607, 0.99064606, 0.98235352,
        0.60681072, 0.83735914, 0.99377093, 0.7437581 , 0.45039655,
        0.91187788, 0.67782276, 0.68578058, 0.52439113, 0.11889323,
        0.67328874, 0.52116607, 0.9436236 , 0.73302253, 0.93126535,
        0.89421599, 0.85889565, 0.02622049, 0.70675434, 0.47793682,
        0.96954189, 0.52116607, 0.97805946, 0.02

In [113]:
pred.mean()

0.9337748344370861

In [165]:
pred = lr.predict(x_train[features])
(pred == y_train.values).mean(), y_train.values.mean()

(0.7523011981478064, 0.7988668555240793)

### NN

In [8]:
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
import tensorflow as tf

In [154]:
model = Sequential([
    InputLayer(input_shape = (len(features), )),
    Dense(32, activation='tanh'),
    Dropout(0.7),
    Dense(32, activation='tanh'),
    Dropout(0.7),
    Dense(1, activation=tf.keras.activations.softmax)
])

In [155]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy())

In [156]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 32)                384       
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_17 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 33        
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________


In [157]:
model.fit(x_train[features], y_train, validation_data = (x_val[features], y_val),
         epochs = 500)

Epoch 1/500
23/23 [==============================] - 1s 6ms/step - loss: 0.9681 - val_loss: 0.5960
Epoch 2/500
23/23 [==============================] - 0s 2ms/step - loss: 0.8428 - val_loss: 0.4852
Epoch 3/500
23/23 [==============================] - 0s 2ms/step - loss: 0.6913 - val_loss: 0.4790
Epoch 4/500
23/23 [==============================] - 0s 2ms/step - loss: 0.6389 - val_loss: 0.4938
Epoch 5/500
23/23 [==============================] - 0s 1ms/step - loss: 0.6235 - val_loss: 0.5130
Epoch 6/500
23/23 [==============================] - 0s 1ms/step - loss: 0.6122 - val_loss: 0.5168
Epoch 7/500
23/23 [==============================] - 0s 1ms/step - loss: 0.5957 - val_loss: 0.5258
Epoch 8/500
23/23 [==============================] - 0s 1ms/step - loss: 0.6086 - val_loss: 0.5309
Epoch 9/500
23/23 [==============================] - 0s 1ms/step - loss: 0.5475 - val_loss: 0.5231
Epoch 10/500
23/23 [==============================] - 0s 2ms/step - loss: 0.5454 - val_loss: 0.5155
Epoch 11/

23/23 [==============================] - 0s 1ms/step - loss: 0.3920 - val_loss: 0.3951
Epoch 84/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3797 - val_loss: 0.3951
Epoch 85/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3807 - val_loss: 0.3938
Epoch 86/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3929 - val_loss: 0.3928
Epoch 87/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3902 - val_loss: 0.3930
Epoch 88/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3991 - val_loss: 0.3929
Epoch 89/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3985 - val_loss: 0.3881
Epoch 90/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3787 - val_loss: 0.3909
Epoch 91/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3926 - val_loss: 0.3854
Epoch 92/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3840 - val_loss: 0.3931
Epoch 93/500


23/23 [==============================] - 0s 2ms/step - loss: 0.3414 - val_loss: 0.3489
Epoch 165/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3549 - val_loss: 0.3489
Epoch 166/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3538 - val_loss: 0.3696
Epoch 167/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3677 - val_loss: 0.3639
Epoch 168/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3633 - val_loss: 0.3389
Epoch 169/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3587 - val_loss: 0.3587
Epoch 170/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3532 - val_loss: 0.3501
Epoch 171/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3579 - val_loss: 0.3544
Epoch 172/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3427 - val_loss: 0.3475
Epoch 173/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3332 - val_loss: 0.3414
Epoc

23/23 [==============================] - 0s 2ms/step - loss: 0.3457 - val_loss: 0.3442
Epoch 246/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3328 - val_loss: 0.3450
Epoch 247/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3266 - val_loss: 0.3569
Epoch 248/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3306 - val_loss: 0.3489
Epoch 249/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3368 - val_loss: 0.3538
Epoch 250/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3235 - val_loss: 0.3450
Epoch 251/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3224 - val_loss: 0.3485
Epoch 252/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3347 - val_loss: 0.3577
Epoch 253/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3310 - val_loss: 0.3599
Epoch 254/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3327 - val_loss: 0.3459
Epoc

23/23 [==============================] - 0s 1ms/step - loss: 0.3136 - val_loss: 0.3503
Epoch 327/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3312 - val_loss: 0.3412
Epoch 328/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3315 - val_loss: 0.3485
Epoch 329/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3419 - val_loss: 0.3453
Epoch 330/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3383 - val_loss: 0.3435
Epoch 331/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3164 - val_loss: 0.3567
Epoch 332/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3464 - val_loss: 0.3705
Epoch 333/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3452 - val_loss: 0.3475
Epoch 334/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3279 - val_loss: 0.3464
Epoch 335/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3424 - val_loss: 0.3514
Epoc

23/23 [==============================] - 0s 1ms/step - loss: 0.3459 - val_loss: 0.3628
Epoch 408/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3281 - val_loss: 0.3483
Epoch 409/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3241 - val_loss: 0.3300
Epoch 410/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3338 - val_loss: 0.3354
Epoch 411/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3328 - val_loss: 0.3263
Epoch 412/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3280 - val_loss: 0.3226
Epoch 413/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3160 - val_loss: 0.3312
Epoch 414/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3303 - val_loss: 0.3239
Epoch 415/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3272 - val_loss: 0.3279
Epoch 416/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3096 - val_loss: 0.3383
Epoc

23/23 [==============================] - 0s 1ms/step - loss: 0.3165 - val_loss: 0.3307
Epoch 489/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3320 - val_loss: 0.3224
Epoch 490/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3303 - val_loss: 0.3210
Epoch 491/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3194 - val_loss: 0.3348
Epoch 492/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3321 - val_loss: 0.3224
Epoch 493/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3194 - val_loss: 0.3138
Epoch 494/500
23/23 [==============================] - 0s 1ms/step - loss: 0.3114 - val_loss: 0.3239
Epoch 495/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3140 - val_loss: 0.3198
Epoch 496/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3122 - val_loss: 0.3235
Epoch 497/500
23/23 [==============================] - 0s 2ms/step - loss: 0.3179 - val_loss: 0.3292
Epoc

In [158]:
pred = model.predict(x_val[features])
(pred == y_val.values).mean(), y_val.mean()

(0.7814569536423841,
 liked    0.781457
 dtype: float64)

In [159]:
pred.mean()

1.0

In [160]:
pred = model.predict(x_train[features])
(pred == y_train.values).mean(), y_train.mean()

(0.7988668555240793,
 liked    0.798867
 dtype: float64)

In [161]:
pred.mean()

1.0

In [4]:
from gen_atr import *

In [5]:
pr = PodcastRecommendation(uri, (user, psw))

In [11]:
df = pr.gen_df('6C561484AED5C02')

In [6]:
#pr.gen_data(df)

In [7]:
pr.recommend('6C561484AED5C02')

47    b4c3c3ebdd76e284f7d9fa358ac82030
17    c9add5e9e81a4b3ca963adab5b87083f
7     a37fb116709bfdb2dd58ea4f784cb815
42    a3a535f66c7e8004e7dc54c2b2829a9e
43    b70d658c901897359bb848cf876cbcbc
14    f9938eec02c773585575a091fe35c425
2     d6ef1df72820e4fc2591f8c3a295b1b7
6     c4b4a343d1131676bf592d266629d6dc
18    d2042758dcacc179d98a76d7673065ac
13    bd1eeaa544589b34fe3056a7a60d9360
4     a627d4945f6fcfc2c9050233cbb5369a
16    bffe9d51afd8a6c6a89c7cbb06da788f
1     f81c8201b9e4f47961987e287ac54cc0
3     db45e4be97084c8d133d88cee1c97897
20    b61de95fa84608350836ed76b0243eef
12    d2da82feac2f430eb12a29a5ca22d0c5
48    a5416c179e94a74e61827c88e405b083
25    f42b4160ff9b8933d74c411ee40596ea
5     a145b05c1f310d835b6086fdb03493a9
44    fa3cac03e05c1e9ec792e95e4850d7ce
41    c1adb6ca5ca39575420fda03c099b037
49    bda31244e9514743575bfd63b1c2aa6e
8     ed1d26a2f16587ec292421e188f8ee83
45    d87a09dc72d2914ab0ed9591d13b5ff3
29    e853203063767cd03182118755f515e5
46    abbdecf5e1cde3bb520